<h1 class='font-effect-3d' style='font-family:Ewert; color:#ff355e;'>📑 Styling, Links and Modules</h1>

---

#### [🌐 tf.keras](https://www.tensorflow.org/api_docs/python/tf/keras)
#### [🌐 The Data Set of Flower Images (R)](https://www.kaggle.com/olgabelitskaya/the-data-set-of-flower-images-r) & [🌐 Github Repository](https://github.com/OlgaBelitskaya/deep_learning_projects/tree/master/DL_PP0) & [🌐 Google Colaboratory Version](https://colab.research.google.com/drive/1H2ArWH_1kYfkIoCbxleX-aHAozRVBAdB)

In [ ]:
%%html
<style> 
@import url('https://fonts.googleapis.com/css?family=Ewert|Roboto&effect=3d'); 
a,h4 {color:slategray; font-family:Roboto; text-shadow:3px 3px 3px #aaa;}
span {color:black; font-family:Roboto; text-shadow:3px 3px 3px #aaa;}
div.output_prompt,div.output_area pre {color:slategray;}
div.input_prompt,div.output_subarea {color:#ff355e;}      
div.output_stderr pre {background-color:gainsboro;}  
div.output_stderr {background-color:slategrey;}     
</style>

In [ ]:
import warnings; warnings.filterwarnings('ignore')
import pandas as pd,numpy as np,pylab as pl
import h5py,cv2,os,tensorflow as tf
import tensorflow.keras.preprocessing.image as tkimg
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import \
MaxPooling2D,Conv2D,Dense,LSTM,PReLU,LeakyReLU,\
BatchNormalization,Flatten,Activation,Dropout,\
GlobalMaxPooling2D,GlobalAveragePooling2D
from tensorflow.keras.callbacks import \
ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.preprocessing.image \
import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.inception_v3 \
import InceptionV3
from keras.applications.vgg16 import VGG16
files_path='../input/flower-color-images/'
fw='/checkpoints'
print('tensorflow version:',tf.__version__)

In [ ]:
def keras_history_plot(fit_history,fig_size,color='#ff355e'):
    keys=list(fit_history.history.keys())
    list_history=[fit_history.history[keys[i]] 
                  for i in range(len(keys))]
    dfkeys=pd.DataFrame(list_history).T
    dfkeys.columns=keys
    fig=pl.figure(figsize=(fig_size,fig_size))
    ax1=fig.add_subplot(2,1,1)
    dfkeys.iloc[:,[0,2]].plot(
        ax=ax1,color=['slategray',color],grid=True)
    ax2=fig.add_subplot(2,1,2)
    dfkeys.iloc[:,[1,3]].plot(
        ax=ax2,color=['slategray',color],grid=True)
    pl.tight_layout(); pl.show()
def cb(fw):
    early_stopping=EarlyStopping(
        monitor='val_loss',patience=20,verbose=2)
    checkpointer=ModelCheckpoint(
        filepath=fw,save_best_only=True,verbose=2,
        save_weights_only=True,monitor='val_accuracy',mode='max')
    lr_reduction=ReduceLROnPlateau(
        monitor='val_loss',verbose=2,patience=5,factor=.8)
    return [checkpointer,early_stopping,lr_reduction]

In [ ]:
#tpu=tf.distribute.cluster_resolver.TPUClusterResolver()
#tf.config.experimental_connect_to_cluster(tpu)
#tf.tpu.experimental.initialize_tpu_system(tpu)
#tpu_strategy=tf.distribute.experimental.TPUStrategy(tpu)
#with tpu_strategy.scope():

<h1 class='font-effect-3d' style='font-family:Ewert; color:#ff355e'>📑 Data Exploration</h1>

In [ ]:
with h5py.File(files_path+\
               'FlowerColorImages.h5','r') as f:
    keys=list(f.keys())
    images=np.array(f[keys[0]],dtype='float32')/255
    labels=np.array(f[keys[1]],dtype='int32')
    f.close()
names=['phlox','rose','calendula','iris',
       'max chrysanthemum','bellflower','viola',
       'rudbeckia laciniata','peony','aquilegia']
fig=pl.figure(figsize=(9,7))
n=np.random.randint(1,30)
for i in range(n,n+8):
    ax=fig.add_subplot(2,4,i-n+1,\
    xticks=[],yticks=[],title=names[labels[i]])
    ax.imshow((images[i]))
st1='Images` Array: %s'
st2='Labels` Array: %s'
print(st1%str(images.shape))
print(st2%str(labels.shape))

In [ ]:
x_train,x_test,y_train,y_test=\
train_test_split(images,labels,test_size=.1,random_state=1)
m=int(len(x_test)/2)
x_valid,y_valid=x_test[:m],y_test[:m]
x_test,y_test=x_test[m:],y_test[m:]
print([x_train.shape,x_test.shape,x_valid.shape,
       y_train.shape,y_test.shape,y_valid.shape])

<h1 class='font-effect-3d' style='font-family:Ewert; color:#ff355e'>📑 Keras Applications</h1>

#### InceptionV3

In [ ]:
steps,epochs=16,15
iv3_model,fr_history,unfr_history=[],[],[]
def inceptionv3_train():
    global steps,epochs,iv3_model,fr_history,unfr_history
    iv3base_model=InceptionV3(
        weights='imagenet',include_top=False)
    x=iv3base_model.output
    x=GlobalAveragePooling2D()(x)
    x=Dense(512,activation='relu')(x)
    x=LeakyReLU(alpha=.02)(x)
    x=Dropout(.25)(x)        
    x=Dense(64)(x)
    x=LeakyReLU(alpha=.02)(x)
    x=Dropout(.25)(x)
    y=Dense(10,activation='softmax')(x)
    iv3_model=Model(inputs=iv3base_model.input,outputs=y)
    # freezing convolutional layers
    for layer in iv3base_model.layers:
        layer.trainable=False
    iv3_model.compile(loss='sparse_categorical_crossentropy',
                      optimizer='nadam',metrics=['accuracy'])
    early_stopping=EarlyStopping(
        monitor='val_loss',patience=20,verbose=2)
    checkpointer=ModelCheckpoint(
        filepath=fw,save_best_only=True,verbose=2,
        save_weights_only=True,monitor='val_accuracy',mode='max')
    lr_reduction=ReduceLROnPlateau(
        monitor='val_loss',verbose=2,patience=5,factor=.8)
    data_generator=ImageDataGenerator(
        shear_range=.2,zoom_range=.2,horizontal_flip=True)
    fr_history=iv3_model.fit_generator(data_generator.flow(
        x_train,y_train,batch_size=12),
        steps_per_epoch=steps,epochs=epochs,verbose=2, 
        validation_data=(x_valid,y_valid),
        callbacks=[checkpointer,early_stopping,lr_reduction])
    #for layer in iv3_model.layers[:173]:
    #    layer.trainable=False
    # unfreezing the layers [173:]
    for layer in iv3_model.layers:
         layer.trainable=True   
    iv3_model.compile(loss='sparse_categorical_crossentropy',
                      optimizer='nadam',metrics=['accuracy'])
    unfr_history=iv3_model.fit_generator(data_generator.flow(
        x_train,y_train,batch_size=12),verbose=2,
        steps_per_epoch=steps,epochs=epochs,
        callbacks=[checkpointer,early_stopping,lr_reduction],
        validation_data=(x_valid,y_valid))

In [ ]:
inceptionv3_train()

In [ ]:
keras_history_plot(fr_history,7)
keras_history_plot(unfr_history,7)
iv3_model.load_weights(fw)
iv3_model.evaluate(x_test,y_test,verbose=0)

In [ ]:
for i,layer in enumerate(iv3_model.layers):
    print(i,layer.name)

In [ ]:
y_test_predict=np.argmax(iv3_model.predict(x_test),axis=1)
fig=pl.figure(figsize=(9,7))
randch=np.random.choice(x_test.shape[0],size=8,replace=False)
for i,idx in enumerate(randch):
    ax=fig.add_subplot(2,4,i+1,xticks=[],yticks=[])
    ax.imshow(np.squeeze(x_test[idx]))
    pred_idx=y_test_predict[idx]
    true_idx=y_test[idx]
    ti='{}\n({})'.format(names[pred_idx],names[true_idx])
    ax.set_title(ti,color=(
        'slategray' if pred_idx==true_idx else '#ff355e'))
pl.tight_layout(); pl.show()

#### VGG16

In [ ]:
vgg16base_model=VGG16(weights='imagenet',
                      include_top=False)
pvx_train=vgg16base_model.predict(x_train)
pvx_valid=vgg16base_model.predict(x_valid)
pvx_test=vgg16base_model.predict(x_test)
sh=pvx_train.shape[1:]

In [ ]:
def vgg16model():
    model=Sequential()  
    model.add(GlobalAveragePooling2D(input_shape=sh))   
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=.02))
    model.add(Dropout(.5))        
    model.add(Dense(64))
    model.add(LeakyReLU(alpha=.02))
    model.add(Dropout(.25))   
    model.add(Dense(10,activation='softmax'))    
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='nadam',metrics=['accuracy'])
    return model
vgg16model=vgg16model()

In [ ]:
early_stopping=EarlyStopping(
    monitor='val_loss',patience=20,verbose=2)
checkpointer=ModelCheckpoint(
    filepath=fw,save_best_only=True,verbose=2,
    save_weights_only=True,monitor='val_accuracy',mode='max')
lr_reduction=ReduceLROnPlateau(
    monitor='val_loss',verbose=2,patience=5,factor=.8)
history=vgg16model.fit(
    pvx_train,y_train,validation_data=(pvx_valid,y_valid), 
    epochs=100,batch_size=64,verbose=2, 
    callbacks=[checkpointer,lr_reduction,early_stopping])

In [ ]:
keras_history_plot(history,7)
vgg16model.load_weights(fw)
vgg16model.evaluate(pvx_test,y_test,verbose=0)

In [ ]:
y_test_predict=np.argmax(vgg16model.predict(pvx_test),axis=1)
fig=pl.figure(figsize=(9,7))
randch=np.random.choice(x_test.shape[0],size=8,replace=False)
for i,idx in enumerate(randch):
    ax=fig.add_subplot(2,4,i+1,xticks=[],yticks=[])
    ax.imshow(np.squeeze(x_test[idx]))
    pred_idx=y_test_predict[idx]
    true_idx=y_test[idx]
    ti='{}\n({})'.format(names[pred_idx],names[true_idx])
    ax.set_title(ti,color=(
        'slategray' if pred_idx==true_idx else '#ff355e'))
pl.tight_layout(); pl.show()